# Gemini Sentiment Analysis Agent Demo


### Get API keys

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in [Google AI Studio](https://makersuite.google.com/app/apikey).  You'll also need an [OpenAI API key](https://openai.com/).

Once you have your API keys create the following Google Colab "secrets"
`GOOGLE_API_KEY` and `OPENAI_API_KEY` which will contain your respective keys.

## Initial Setup

In [1]:
!pip install llama-index llama-hub google-generativeai openai pypdf youtube_transcript_api -q

In [2]:
!pip install trulens_eval ipywidgets -q

In [3]:
import os
import openai

openai.api_key = os.environ.get('OPENAI_API_KEY')

In [4]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO, force=True)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

### Vectorize Content For RAG

In [5]:
import os.path
from llama_index import (
  VectorStoreIndex,
  SimpleDirectoryReader,
  StorageContext,
  load_index_from_storage,
  )
from llama_hub.youtube_transcript import YoutubeTranscriptReader

# Attempt to load embeddings
try:
  storage_context = StorageContext.from_defaults(
      persist_dir="./storage/challenging_child"
  )
  challenging_child_index = load_index_from_storage(storage_context)

  storage_context = StorageContext.from_defaults(
      persist_dir="./storage/mindfulness_TB_50"
  )
  mindfulness_TB_50_index = load_index_from_storage(storage_context)

  storage_context = StorageContext.from_defaults(
      persist_dir="./storage/mindfulness_TB_relationships"
  )
  mindfulness_TB_relationships_index = load_index_from_storage(storage_context)

except:
  # Need to generate embeddings
  challenging_child_docs = SimpleDirectoryReader(
      input_files=["./data/The_Challenging_Child_Toolbox.pdf"]
  ).load_data(show_progress=True)
  mindfulness_TB_50_docs = SimpleDirectoryReader(
      input_files=["./data/The_Mindfulness_Toolbox__50_Practical_Tips_Tools.pdf"]
  ).load_data(show_progress=True)
  mindfulness_TB_relationships_docs = SimpleDirectoryReader(
      input_files=["./data/The_Mindfulness_Toolbox_for_Relationships.pdf"]
  ).load_data(show_progress=True)

  # Build index
  challenging_child_index = VectorStoreIndex.from_documents(challenging_child_docs)
  mindfulness_TB_50_index = VectorStoreIndex.from_documents(mindfulness_TB_50_docs)
  mindfulness_TB_relationships_index = VectorStoreIndex.from_documents(mindfulness_TB_relationships_docs)

  # Persist
  challenging_child_index.storage_context.persist(persist_dir="./storage/challenging_child")
  mindfulness_TB_50_index.storage_context.persist("./storage/mindfulness_TB_50")
  mindfulness_TB_relationships_index.storage_context.persist(persist_dir="./storage/mindfulness_TB_relationships")

INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.


In [6]:
challenging_child_engine = challenging_child_index.as_query_engine()
mindfulness_TB_50_engine = mindfulness_TB_50_index.as_query_engine()
mindfulness_TB_relationships_engine = mindfulness_TB_relationships_index.as_query_engine()

### Test to make sure our data loaded

In [7]:
from IPython.display import Markdown

In [8]:
# The_Challenging_Child_Toolbox
query_engine = challenging_child_index.as_query_engine()
response = query_engine.query("What is the definition of a challenging child?")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

A challenging child refers to a child who has emotional and behavioral difficulties. These difficulties may require special attention and support from caregivers or professionals working with them.

In [9]:
# The_Mindfulness_Toolbox__50_Practical_Tips_Tools
query_engine = mindfulness_TB_50_index.as_query_engine()
response = query_engine.query("What are the benefits of a mindfulness practice?")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Mindfulness practice has several benefits. It enhances flexibility and adaptability, allowing individuals to break free from old habits and retrain their brains. It cultivates curiosity and greater ease, encouraging exploration of the journey and process rather than being overly focused on the outcome. Mindfulness also changes one's relationship to self-critical and self-blaming thoughts, promoting greater patience, kindness, acceptance, and hospitality towards oneself and others. Additionally, mindfulness encourages greater fulfillment in daily life by focusing on the present moment, reducing rumination and negative thoughts, as well as anxiety about the future.

In [10]:
# The_Mindfulness_Toolbox_for_Relationships
query_engine = mindfulness_TB_relationships_index.as_query_engine()
response = query_engine.query("How does mindfulness relate to relationships?")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Mindfulness relates to relationships by engaging a sense of curiosity and openness, which helps individuals escape the ego-centric perspective and view others in a new way. It allows individuals to relate at a profound level to the inner and outer environments of their lives. By practicing mindfulness, individuals can bring awareness, compassion, and openness into their relationships, leading to a sense of liberation and freedom from old habits and toxic ways of reacting. Mindfulness also helps dissolve the illusion of separateness and cultivates a sense of empathy, compassion, trust, and acceptance, which are essential for healthy relationships.

## Define Agent Tools

### Analyze image tool

In [11]:
from llama_index.tools import BaseTool, FunctionTool
from llama_index.multi_modal_llms.gemini import GeminiMultiModal
from llama_index.multi_modal_llms.generic_utils import load_image_urls
from typing import List

In [12]:
def analyze_image(img_urls: List[str]) -> str:
  """Calls our Gemini vision API to analyze the image and return a description of the text contained in the image.
      Returns: A string with a description of the image and the mood it conveys if any.

  Args:
      img_urls (List[str]): The URL of one or more images that convey the users mood
  """
  image_documents = load_image_urls(img_urls)

  gemini = GeminiMultiModal(model="models/gemini-pro-vision", api_key=userdata.get('GOOGLE_API_KEY'))

  complete_response = gemini.complete(
      prompt="Identify what you see in the image and what mood it conveys if any",
      image_documents=image_documents,
  )

  return complete_response

vision_tool = FunctionTool.from_defaults(fn=analyze_image)

### Save session tool

In [13]:
def save_session(chat_summary: str) -> bool:
  """Persists a summary of the user's chat history. Use this tool when the user is happy with your recomendations and done with the session
      Returns: A boolean saying if the chat history was persisted

  Args:
      chat_summary (str): A summary of the chat history, including the users name if it was provided in the chat session
  """

  # Here is where we would persist the chat summary so we can retrive it when we start a new sessions

  return True

save_tool = FunctionTool.from_defaults(fn=save_session)

### Mindfulness routine recomendation tool

Use Gemini Pro to examine the transcript of our mindfulness videos

In [14]:
from llama_hub.youtube_transcript import YoutubeTranscriptReader

# Download video transcripts
loader = YoutubeTranscriptReader()
documents = loader.load_data(ytlinks=['https://youtu.be/nPvN1OI7h80?si=iB3TYXI-9Ztc0Qpi',
                                      'https://youtu.be/8ZhjZD8rj3E?si=n30dJ0D55tRl9frP',
                                      'https://youtu.be/mGoGYu7F2PA?si=qsvemuKLDdlnvC6z',
                                      'https://youtu.be/d0VZk3Dd0nw?si=yb3Zr4XGErHSENFB',
                                      'https://youtu.be/mme5NC0F7wQ?si=Dd_KECuQCjPJSFXu',
                                      'https://youtu.be/iPjFd1eL40Y?si=3RI6N1mw6J3unntk'
                                      'https://youtu.be/iPjFd1eL40Y?si=bC0hbYPR0jbVRim9',
                                      'https://youtu.be/zVbRxs_QFBA?si=Bn7MC5yG2QcXtHZP',
                                      'https://youtu.be/6xTx984jSFc?si=J3rNB2v9OUvAEBsx',
                                      'https://youtu.be/1qBbuKWWTGY?si=GO7hyw9za62_V9cB',
                                      'https://youtu.be/fj0dh_KxIg4?si=dxVvRm6AC3Ixls1b',
                                      'https://youtu.be/4ksAYqRku-s?si=q3cJ7Spvbgi6H2gi',
                                      'https://youtu.be/f2ZwrQF6VQM?si=ujQw2fq5Ww3_b9Mv',
                                      'https://youtu.be/IL_1DRZDzWc?si=-RutJc161ZO5wgO4'])

In [18]:
from llama_index.llms import Gemini

videos = {}

# Have Gemini analyze the transcript sentiment and who this video is for
for document in documents:
  response = Gemini(model='models/gemini-pro', api_key=os.environ.get('GOOGLE_API_KEY')).complete(f"Summarize the transcript from this mindfulness video and who should use it: {document.text}")
  videos[document.metadata['video_id']] = response.text


Create a tool which recomends a mindfulness routine based on how the user is feeling

In [19]:
import json

json_string = json.dumps(videos)
json_string

'{"nPvN1OI7h80": "**Summary of the Mindfulness Video:**\\n\\nThe video guides viewers through a mindfulness meditation to help them connect with their younger selves and address feelings of loneliness and misunderstanding within their families. It encourages viewers to create a safe and peaceful place in their minds, where they can meet their younger selves and offer comfort and understanding. The meditation emphasizes the importance of self-acceptance and celebrating one\'s unique values and beliefs, even if they differ from those of their family.\\n\\n**Who Should Use This Video:**\\n\\n- Individuals who feel lonely or misunderstood within their families.\\n- People who struggle with self-acceptance and self-esteem due to family dynamics.\\n- Those who want to develop a stronger connection with their inner selves and find comfort in their own company.\\n- Anyone seeking a guided meditation to promote self-compassion and resilience.", "8ZhjZD8rj3E": "**Summary of the Mindfulness Video

In [20]:
import json


def recomend_mindfulness(feelings_summary: str) -> str:
  """Recomends a mindfullness routine based on how the user is feeling
      Returns: A string with a description of the mindfulness routine it recomends and a link to the youtube video

  Args:
      feelings_summary (str): A summary of the user's feeings
  """
  videos_string = json.dumps(videos)

  response = Gemini(model='models/gemini-pro', api_key=os.environ.get('GOOGLE_API_KEY')).complete(f"""
    I'm sending you a json with a list of youtube mindfulness videos
    The key is the youtube video id and the value is a summary of the video transcript
    from this mindfulness video and who should use it.  The json is here: {videos_string}.
    Recomend a video based on my feelings and include the youtube link in the format https://www.youtube.com/watch?v=video_id
    Here is a summary of the user's feelings: {feelings_summary}""")

  return response.text

recommend_tool = FunctionTool.from_defaults(fn=recomend_mindfulness)

### Query engine tool

In [21]:
from llama_index.tools import QueryEngineTool, ToolMetadata

query_engine_tools = [
    QueryEngineTool(
        query_engine=challenging_child_engine,
        metadata=ToolMetadata(
            name="challenging_child",
            description=(
                "The Challenging Child Toolbox 75 Mindfulness Based Practices Tools and Tips for Therapists"
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=mindfulness_TB_50_engine,
        metadata=ToolMetadata(
            name="mindfulness_TB_50",
            description=(
                "The Mindfulness Toolbox 50 Practical Tips Tools Handouts for Anxiety Depression Stress and Pain"
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=mindfulness_TB_relationships_engine,
        metadata=ToolMetadata(
            name="mindfulness_TB_relationships",
            description=(
                "The Mindfulness Toolbox for Relationships 50 Practical Tips Tools Handouts for Building Compassionate Connections"
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

In [36]:
# tools = query_engine_tools + [vision_tool, save_tool, recommend_tool]
# only using query_engine_tools for trulens test
tools = query_engine_tools

## Create LlamaIndex Agent

### System prompt to give our agent some personality

In [60]:
# SYSTEM_PROMPT = """You are an emotional support assistant with the expertise of an experienced counselor. Your primary role is to assist the user by encouraging them to provide a drawing that conveys their mood and then recomending a mindfulness routine.
# You offer professional, friendly, and helpful guidance based on current counseling and mindfulness practices. Once you receive the image, interpret it using your tools to discern what the user might be feeling and confirm with them if your observation is correct.
# If your interpretation does not align with their feelings, engage in a dialogue until you accurately understand their mood. Your knowledge is exclusively focused on understanding the user's emotions and recommending mindfulness routines using your tools tools, tailored to their mood.
# Your responses should be limited to one sentence when possible so the user doesn't have to do a lot of reading.  An example response is "For feels of {insert feelings here} I recomend this mindfulness routine {insert link here}.
# Thus, you will only provide responses related to these areas. If a question falls outside your area of expertise or if you lack the necessary information, you will inform the user by saying,
# 'Sorry, I do not know the answer to your question.' and then prompt for more information related to their feelings."""
SYSTEM_PROMPT = """You are an emotional support assistant with the expertise of an experienced counselor.
You offer professional, friendly, and helpful guidance based on current counseling and mindfulness practices. Your knowledge is exclusively focused on understanding the user's emotions and recommending mindfulness routines using your tools, tailored to their mood.
Keep the answer short where possible.
Thus, you will only provide responses related to these areas. If a question falls outside your area of expertise or if you lack the necessary information, you will inform the user by saying,
'Sorry, I do not know the answer to your question.' and then prompt for more information related to their feelings."""

### Init our agent

Using OpenAIAgent

In [61]:
from llama_index.agent import OpenAIAgent
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-4-1106-preview") # Using GPT-4 Turbo (Beta)

agent = OpenAIAgent.from_tools(
    tools,
    llm=llm,
    verbose=False,
    system_prompt=SYSTEM_PROMPT,
)

Alternative approach using ReActAgent so we can use Gemini as our LLM.  ReActAgent is designed to allow alternatives to OpenAI.

NOTE: This never worked reliably with Gemini because it's not conversational.  If you say 'Hi' it trys to ue a tool rather than just chatting.  Would be interesting to try it with other LLMs.  I tried using GPT-4 Turbo and it seemed to work.

In [187]:
# from llama_index.llms import Gemini, ChatMessage
# from llama_index.agent import AgentRunner, ReActAgentWorker, ReActAgent

# llm = Gemini(model="models/gemini-pro", api_key=userdata.get('GOOGLE_API_KEY'), temperature=0.1)
# agent = ReActAgent.from_tools(tools, llm=llm, verbose=True, chat_history=[ChatMessage(role="user", content=SYSTEM_PROMPT)])

## Test Our Agent

I'm picturing the frontend give the user the option to chat or attach one or more images that would get uploaded to cloud storage.  Then our back could prompt the agent with something like "Here's a drawing of how I'm feeling {Image URL Here}."

For the demo below you need to manually paste the image URL into the chat (i.e. https://ih1.redbubble.net/image.3636044620.1142/bg,f8f8f8-flat,750x,075,f-pad,750x1000,f8f8f8.u5.jpg)

Single prompt example

In [213]:
response = agent.chat("hi I'm Andrew")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Hello Andrew! How can I assist you today? If you're feeling up to it, you can share a drawing that conveys your mood, and I can help you with a mindfulness routine based on that.

Multi-turn chat example

In [ ]:
response = agent.chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.

Human: https://ih1.redbubble.net/image.3636044620.1142/bg,f8f8f8-flat,750x,075,f-pad,750x1000,f8f8f8.u5.jpg
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:tornado.access:200 GET /v1beta/models/gemini-pro-vision?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 481.80ms
200 GET /v1beta/models/gemini-pro-vision?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 481.80ms
INFO:tornado.access:200 POST /v1beta/models/gemini-pro-vision:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 13925.80ms
200 POST /v1beta/models/gemini-pro-vision:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 13925.80ms
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Assistant: The drawing suggests you

## Reset Agent State

The agent's knowledge is based on it's chat history which we can easily reset.

In [190]:
agent.reset()
agent.chat_history

[]

If we ask it our name it shouldn't know

In [191]:
response = agent.chat("whats my name?")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Sorry, I do not know the answer to your question. Could you share with me how you're feeling today or provide a drawing that conveys your mood?

Let's tell it our name and ask again

In [192]:
response = agent.chat("hi I'm Andrew")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Hello Andrew! How are you feeling today? If you'd like, you can express your mood by sharing a drawing with me.

In [193]:
response = agent.chat("whats my name?")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Your name is Andrew. How can I assist you with your emotions today, Andrew? Would you like to share a drawing that reflects how you're feeling?

It knows our name and we can see the chat history

In [39]:
agent.chat_history

[]

As a sanity check lets reset the chat history one more time and ask it our name again.

In [40]:
agent.reset()
agent.chat_history

[]

In [196]:
response = agent.chat("whats my name?")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Sorry, I do not know the answer to your question. If you're feeling up to it, would you like to share a drawing that conveys your mood? This can help me understand how you're feeling and recommend a mindfulness routine tailored to you.

## Handle Returning User

When returning to the conversation we can start with a chat history, even if the agent was reset, so the agent knows what we talked about last.

As a sanity check lets first reset out chat history and make sure it doesn't remember us.

In [197]:
agent.reset()
agent.chat_history

[]

In [198]:
response = agent.chat("whats my name?")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Sorry, I do not know the answer to your question. Could you share with me how you're feeling today? If you're comfortable, you can express your mood through a drawing and share it with me.

It doesn't know our name.  Now let's pass a chat history with our name and a summary of our last session that we persisted when the last session ended.

In [25]:
from llama_index.llms import ChatMessage

persisted_chat_history = "Andrew shared a drawing that conveyed feelings of sadness and loneliness. After confirming his mood, a mindfulness exercise was recommended to help him feel more grounded and connected."
chat_history = [ChatMessage(role="user", content=persisted_chat_history, additional_kwargs={})]

In [26]:
response = agent.chat("I'm a returning user.  Address me by name and ask how I'm doing since we last spoke.  Also reiterate how I was last feeling so I know you understand.", chat_history=chat_history)
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Hello Andrew, it's good to see you back. How have you been doing since we last spoke about the feelings of sadness and loneliness you were experiencing?

# Testing out With TruLens

## Initialize Feedback Function(s)

In [41]:
import numpy as np
from trulens_eval import Feedback, Tru, TruLlama
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.openai import OpenAI

tru = Tru()
# Initialize provider class
openai = OpenAI()

grounded = Groundedness(groundedness_provider=OpenAI())

# Define a groundedness feedback function
f_groundedness = Feedback(grounded.groundedness_measure_with_cot_reasons).on(
    TruLlama.select_source_nodes().node.text.collect()
    ).on_output(
    ).aggregate(grounded.grounded_statements_aggregator)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai.relevance).on_input_output()

# Question/statement relevance between question and each context chunk.
f_qs_relevance = Feedback(openai.qs_relevance).on_input().on(
    TruLlama.select_source_nodes().node.text
    ).aggregate(np.mean)

✅ In groundedness_measure_with_cot_reasons, input source will be set to __record__.app.query.rets.source_nodes[:].node.text.collect() .
✅ In groundedness_measure_with_cot_reasons, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In qs_relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In qs_relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


## Instrument app for logging with TruLens

In [68]:
tru_query_engine_recorder = TruLlama(agent,
    app_id='Willow_Alphav5',
    feedbacks=[f_groundedness, f_qa_relevance, f_qs_relevance])

INFO:alembic.runtime.migration:Context impl SQLiteImpl.
Context impl SQLiteImpl.
INFO:alembic.runtime.migration:Will assume non-transactional DDL.
Will assume non-transactional DDL.


INFO:trulens_eval.database.sqlalchemy_db:✅ added app Willow_Alphav5
✅ added app Willow_Alphav5
INFO:alembic.runtime.migration:Context impl SQLiteImpl.
Context impl SQLiteImpl.
INFO:alembic.runtime.migration:Will assume non-transactional DDL.
Will assume non-transactional DDL.
INFO:trulens_eval.database.sqlalchemy_db:✅ added feedback definition feedback_definition_hash_60450660632685aa33a4f43f83697287
✅ added feedback definition feedback_definition_hash_60450660632685aa33a4f43f83697287
INFO:alembic.runtime.migration:Context impl SQLiteImpl.
Context impl SQLiteImpl.
INFO:alembic.runtime.migration:Will assume non-transactional DDL.
Will assume non-transactional DDL.
INFO:trulens_eval.database.sqlalchemy_db:✅ added feedback definition feedback_definition_hash_0ba3c816b5dab24ea1171caca1bfb70c
✅ added feedback definition feedback_definition_hash_0ba3c816b5dab24ea1171caca1bfb70c
INFO:alembic.runtime.migration:Context impl SQLiteImpl.
Context impl SQLiteImpl.
INFO:alembic.runtime.migration:Wil

## Explore in a Dashboard

In [70]:
# or as context manager
with tru_query_engine_recorder as recording:
    agent.query("How does mindfulness relate to relationships?")
    agent.query("How would you define patience?")
    agent.query("What are the benefits of a mindfulness practice?")
    agent.query("Should I set limits on my children?")
    agent.query("I feel like me resisting a situation in my life has resulted in more suffering than the condition itself.")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
A new object of type <class 'llama_index.memory.chat_memory_buffer.ChatMemoryBuffer'> at 0x20933dba170 is calling an instrumented method <function ChatMemoryBuffer.put at 0x000002092212E3B0>. The path of this call may be incorrect.
Guessing path of new object is app.memory based on other object (0x20931c64490) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x20907933b80 is calling an instrumented method <function BaseQueryEngine.query at 0x000002092212C9D0>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x20931c678e0) using this function.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:alembic.runtime.migration:Context impl SQLiteImpl.
Context impl SQLiteImpl.
INFO:alembic.runtime.migration:Will assume non-transactional DDL.
Will assume non-transactional DDL.
INFO:trulens_eval.database.sqlalchemy_db:✅ feedback result relevance DONE feedback_result_hash_884fcb38c5ff52f53d7df8d44cbed249
✅ feedback result relevance DONE feedback_result_hash_884fcb38c5ff52f53d7df8d44cbed249
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:alembic.runtime.migration:Context impl SQLiteImpl.
Context impl SQLiteImpl.
INFO:alem

In [ ]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.0.105:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [ ]:
tru.get_records_and_feedback(app_ids=[])[0] # pass an empty list of app_ids to get all

INFO:alembic.runtime.migration:Context impl SQLiteImpl.
Context impl SQLiteImpl.
INFO:alembic.runtime.migration:Will assume non-transactional DDL.
Will assume non-transactional DDL.


,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,relevance,qs_relevance,groundedness_measure_with_cot_reasons,relevance_calls,qs_relevance_calls,groundedness_measure_with_cot_reasons_calls,latency,total_tokens,total_cost
0,LlamaIndex_App1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_531ca89ec256899e128ee0637a51118e,"""What did the author do growing up?""","""The author mentioned that before college, the...",-,"{""record_id"": ""record_hash_531ca89ec256899e128...","{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2023-12-19T13:10:25.871039"", ""...",2023-12-19T13:10:29.347691,0.8,0.45,1.000000,[{'args': {'prompt': 'What did the author do g...,[{'args': {'question': 'What did the author do...,[{'args': {'source': ['![](https://s.turbifycd...,3,2126,0.003196
1,Willow_Alpha,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",OpenAIAgent(llama_index.agent.openai.base),record_hash_e6c435d8b8fafbbf397145a4947ba8bb,"""My son is performing poorly in his school. Wh...","""Sorry, I do not know the answer to your quest...",-,"{""record_id"": ""record_hash_e6c435d8b8fafbbf397...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-19T15:46:27.935078"", ""...",2023-12-19T15:46:38.423575,1.0,NaN,0.000000,[{'args': {'prompt': 'My son is performing poo...,[],"[{'args': {'source': [], 'statement': 'Sorry, ...",3,635,0.008210
2,Willow_Alpha,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",OpenAIAgent(llama_index.agent.openai.base),record_hash_8b87d32a191397ba592467ecbefea7e5,"""My son is performing poorly in his school. Wh...","""Sorry, I do not know the answer to your quest...",-,"{""record_id"": ""record_hash_8b87d32a191397ba592...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-19T15:57:04.715265"", ""...",2023-12-19T15:57:17.143217,1.0,NaN,0.000000,[{'args': {'prompt': 'My son is performing poo...,[],"[{'args': {'source': [], 'statement': 'Sorry, ...",10,630,0.008060
3,Willow_Alpha,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",OpenAIAgent(llama_index.agent.openai.base),record_hash_2f69bf215f535424b8f1e570e95f66c8,"""My son is performing poorly in his school. It...","""I'm sorry to hear that your son's school perf...",-,"{""record_id"": ""record_hash_2f69bf215f535424b8f...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-19T15:59:25.360846"", ""...",2023-12-19T15:59:33.989579,0.2,NaN,0.000000,[{'args': {'prompt': 'My son is performing poo...,[],"[{'args': {'source': [], 'statement': 'I'm sor...",12,660,0.008900
4,Willow_Alpha,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",OpenAIAgent(llama_index.agent.openai.base),record_hash_924f9898ff392225d711366dcf5da684,"""I am having issues with me wife. Can you prov...","""It sounds like you're going through a challen...",-,"{""record_id"": ""record_hash_924f9898ff392225d71...","{""n_requests"": 4, ""n_successful_requests"": 4, ...","{""start_time"": ""2023-12-19T16:02:38.832174"", ""...",2023-12-19T16:03:21.125467,1.0,0.35,0.450000,[{'args': {'prompt': 'I am having issues with ...,[{'args': {'question': 'I am having issues wit...,[{'args': {'source': ['✓ Consider teaching t...,8,2361,0.029024
5,Willow_Alpha_v1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",OpenAIAgent(llama_index.agent.openai.base),record_hash_3ab53b08e3ac7a65ec22c496cda039f6,"""I am having issues with my wife. Can you prov...","""It sounds like you're going through a challen...",-,"{""record_id"": ""record_hash_3ab53b08e3ac7a65ec2...","{""n_requests"": 4, ""n_successful_requests"": 4, ...","{""start_time"": ""2023-12-19T19:05:58.886151"", ""...",2023-12-19T19:06:27.721442,1.0,0.40,0.430000,[{'args': {'prompt': 'I am having issues with ...,[{'args': {'question': 'I am having issues wit...,[{'args': {'source': ['✓ Consider teaching t...,3,2469,

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:alembic.runtime.migration:Context impl SQLiteImpl.
Context impl SQLiteImpl.
INFO:alembic.runtime.migration:Will assume non-transactional DDL.
Will assume non-transactional DDL.
INFO:trulens_eval.database.sqlalchemy_db:✅ feedback result relevance DONE feedback_result_hash_c2c570cb6e59641f3ed8ca093d3c587f
✅ feedback result relevance DONE feedback_result_hash_c2c570cb6e59641f3ed8ca093d3c587f
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:alembic.runtime.migration:Context impl SQLiteImpl.
Context impl SQLiteImpl.
INFO:alembic.runtime.migration:Will assume non-transactional DDL.
Will assume non-transactional DDL.
INFO:trulens_eval.database.sqlalchemy_db:✅ feedback result groundedness_measure